In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold,KFold
from keras import regularizers

In [2]:
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

In [86]:
df1 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R6_yes.csv')
#df1.head(5)
print(df1.shape)

(81, 14)


In [87]:
df2 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R6_no.csv')
df3 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R7_yes.csv')
df4 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R7_no.csv')
df5 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R8_yes.csv')
df6 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R8_no.csv')
df7 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R9_yes.csv')
df8 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R9_no.csv')
df9 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R10_yes.csv')
df10 = pd.read_csv(r'C:\Users\bowei\projectes\clockdrawing\csv\R10_no.csv')

In [88]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10],axis=0)
df.head(10)

,spid,imgspid,dementia,gender,race,intvrage,weight,howtallft,howtallin,wrdimmrc,wrddlyrc,clkdraw,clkimgcl,height
0,10000647,100006476,1,2,1,5,153,5,1,5,3,3,1,154.94
1,10000651,100006516,1,1,1,6,200,5,5,3,0,4,1,165.10
2,10001416,100014166,1,2,3,5,132,5,0,4,1,1,1,152.40
3,10001778,100017786,1,2,2,6,184,5,3,5,1,3,3,160.02
4,10001823,100018236,1,2,3,4,144,5,4,2,0,2,1,162.56
5,10001937,100019376,1,2,3,6,120,5,0,3,1,3,1,152.40
6,10001998,100019986,1,1,1,4,221,5,9,2,1,3,1,175.26
7,10002599,100025996,1,2,1,3,174,5,0,5,4,4,1,152.40
8,10003123,100031236,1,1,1,3,172,5,7,6,2,3,1,170.18
9,10003656,100036566,1,1,2,6,145,5,4,4,2,3,1,162.56


In [89]:
df.shape

(1662, 14)

In [90]:
#reindex of all data after concat
df.reset_index(drop=True, inplace=True)

In [91]:
from tqdm import tqdm
allimg = []  
 
for i in tqdm(range(df.shape[0])):
  # 读取图片
  img = image.load_img(r'C:/Users/bowei/projectes/clockdrawing/img_crop/crop/'+str(df['imgspid'][i])+'.jpg',target_size=(224,224,3))
  img = image.img_to_array(img)  
  img = img/255  
  allimg.append(img)  

# 转换为numpy数组
img_total = np.array(allimg)

100%|█████████████████████████████████████████████████████████████████████████████| 1662/1662 [00:04<00:00, 350.84it/s]


In [92]:
img_total.shape

(1662, 224, 224, 3)

In [93]:
# for softmax
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()  
AD_values = encoder.fit_transform(df['dementia'].values)  
AD_values = np.array([AD_values]).T
#AD_values

In [94]:
# for softmax
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_total=enc.fit_transform(AD_values)
y_total=y_total.toarray()
y_total

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
from numpy.random import seed
seed(2)
tf.random.set_seed(9)

X_train, X_test, y_train, y_test = train_test_split(img_total,y_total,test_size=0.2, random_state=22,stratify=y_total)

from keras.optimizers import adam_v2
adam = adam_v2.Adam(learning_rate=0.00001)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,3),padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding="same"))
#model.add(BatchNormalization(axis=-1))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3),activation='relu',padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.25))
model.add(Dense(16,activation='relu'))
model.add(Dense(2,activation='softmax'))

#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["AUC"])
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.AUC()])
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['AUC'])

#history = model.fit(X_train, y_train, batch_size=32, validation_split=0.15, epochs=15, verbose=1, class_weight={0:1,1:5})
history = model.fit(X_train, y_train, batch_size=16, validation_split=0.2, epochs=40, verbose=1, class_weight={0:1,1:5})

In [ ]:
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
prediction = model.predict(X_test)
prediction1 = (prediction > 0.5).astype("int32")

from sklearn.metrics import f1_score
print(f1_score(y_test, prediction1,average ='micro'))

from sklearn.metrics import precision_score
print(precision_score(y_test, prediction1,average ='micro'))
print(precision_score(y_test, prediction1,average ='macro'))
print(precision_score(y_test, prediction1,average ='weighted'))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction1))

In [ ]:
#auc
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, prediction,average ='micro'))
print(roc_auc_score(y_test, prediction,average ='macro'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp

y_label = np.array(y_test)
y_score = np.array(prediction)
n_classes = 2
 
# 计算每一类的ROC
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_label[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
 
# micro（方法二）
fpr["micro"], tpr["micro"], _ = roc_curve(y_label.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
 
# macro（方法一）
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
 
# Plot all ROC curves
lw=2
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)
 
plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)
 
colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
 
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('multi-calss ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test[:,1], prediction1[:,1]))

#TN|FP
#FN|TP